In [5]:
# api.py
import os
import uuid
import json
import tempfile
import pandas as pd
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse

app = FastAPI()

# Allow React frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

os.makedirs("processed_files", exist_ok=True)

# ----------------------- FILE LOADER -----------------------
def load_file(path):
    ext = path.lower()
    if ext.endswith(".xlsx") or ext.endswith(".xls"):
        xls = pd.ExcelFile(path)
        return {sheet: xls.parse(sheet) for sheet in xls.sheet_names}
    elif ext.endswith(".csv"):
        try:
            return {"Sheet1": pd.read_csv(path, encoding="utf-8")}
        except UnicodeDecodeError:
            return {"Sheet1": pd.read_csv(path, encoding="latin1")}
    else:
        raise Exception("Unsupported file format")

# ----------------------- NAN CLEANER -----------------------
def df_to_json_safe(df):
    """Convert DataFrame to JSON-safe dict (NaN → None)"""
    return df.where(pd.notnull(df), None).to_dict(orient="records")

# ----------------------- DATA OPERATIONS -----------------------
def handle_missing(df, strategy="drop", numeric_strategy="mean"):
    df2 = df.copy()
    for col in df2.columns:
        if df2[col].dtype in ["float64", "int64"]:
            if strategy == "drop":
                df2 = df2[df2[col].notna()]
            elif strategy == "impute":
                if numeric_strategy=="mean":
                    df2[col] = df2[col].fillna(df2[col].mean())
                elif numeric_strategy=="median":
                    df2[col] = df2[col].fillna(df2[col].median())
        else:
            df2[col] = df2[col].fillna("missing")
    return df2

def remove_outliers(df, params={}):
    cols = params.get("columns", df.select_dtypes(include=['number']).columns)
    df2 = df.copy()
    for col in cols:
        if col not in df2.columns: continue
        Q1 = df2[col].quantile(0.25)
        Q3 = df2[col].quantile(0.75)
        IQR = Q3 - Q1
        df2 = df2[(df2[col] >= Q1 - 1.5*IQR) & (df2[col] <= Q3 + 1.5*IQR)]
    return df2

def remove_duplicates(df, params={}):
    subset = params.get("columns")
    df2 = df.copy()
    return df2.drop_duplicates(subset=subset, keep=False).reset_index(drop=True)  # remove all duplicates

def replace_values(df, params={}):
    find = params.get("find")
    replace = params.get("replace")
    if find is None:
        return df
    return df.replace(find, replace)

def normalize_text(df, params={}):
    cols = params.get("columns", df.columns)
    mode = params.get("mode", "lower")
    df2 = df.copy()
    for col in cols:
        if col not in df2.columns: continue
        df2[col] = df2[col].astype(str).str.strip()
        if mode=="lower": df2[col] = df2[col].str.lower()
        elif mode=="upper": df2[col] = df2[col].str.upper()
    return df2

def filter_rows(df, params={}):
    col = params.get("column")
    cond = params.get("condition")
    value = params.get("value")
    if col not in df.columns: return df
    series = df[col]
    try:
        if cond=="equals": return df[series==value]
        if cond=="contains": return df[series.astype(str).str.contains(str(value), na=False)]
        if cond=="greater_than": return df[pd.to_numeric(series, errors="coerce") > float(value)]
        if cond=="less_than": return df[pd.to_numeric(series, errors="coerce") < float(value)]
    except: return df
    return df

def merge_columns(df, params={}):
    cols = params.get("columns")
    new_col = params.get("new_column", "merged_column")
    sep = params.get("separator", " ")
    if not cols or any(col not in df.columns for col in cols): return df
    df2 = df.copy()
    df2[new_col] = df2[cols].astype(str).agg(sep.join, axis=1)
    return df2

def convert_data_types(df, params={}):
    col = params.get("column")
    typ = params.get("new_type")
    if col not in df.columns: return df
    df2 = df.copy()
    try:
        if typ=="int": df2[col]=pd.to_numeric(df2[col], errors="coerce").astype("Int64")
        elif typ=="float": df2[col]=pd.to_numeric(df2[col], errors="coerce")
        elif typ=="datetime": df2[col]=pd.to_datetime(df2[col], errors="coerce")
        elif typ=="string": df2[col]=df2[col].astype(str)
    except: pass
    return df2

# Map operations
OPERATION_FUNCTIONS = {
    "handle_missing": handle_missing,
    "remove_duplicates": remove_duplicates,
    "replace_values": replace_values,
    "normalize_text": normalize_text,
    "filter_rows": filter_rows,
    "merge_columns": merge_columns,
    "convert_data_types": convert_data_types,
    "remove_outliers": remove_outliers
}

# ----------------------- PROCESS FILE -----------------------
def process_file(file_path, operations, priority_ops):
    sheets = load_file(file_path)
    ordered_ops = priority_ops + [op for op in operations if op not in priority_ops]
    output = {}
    stats = {}

    for sheet_name, df in sheets.items():
        df_clean = df.copy()
        initial_rows = len(df_clean)
        initial_missing = df_clean.isna().sum().sum()

        for op in ordered_ops:
            func = OPERATION_FUNCTIONS.get(op)
            if not func: continue
            params = operations.get(op, {})
            df_clean = func(df_clean, params)

        duplicates_removed = initial_rows - len(df_clean)
        missing_after = df_clean.isna().sum().sum()

        output[sheet_name] = df_clean
        stats[sheet_name] = {
            "initial_rows": initial_rows,
            "duplicates_removed": duplicates_removed,
            "initial_missing": initial_missing,
            "missing_after": missing_after,
            "operations_applied": ordered_ops
        }

    first_sheet = list(output.keys())[0]

    return {
        "status": "success",
        "message": "Processed successfully",
        "data": output,
        "preview": df_to_json_safe(output[first_sheet]),
        "sheet_names": list(output.keys()),
        "stats": stats
    }

# ----------------------- API ENDPOINTS -----------------------
@app.post("/process")
async def process_file_api(
    file: UploadFile = File(...),
    operations: str = Form(...),
    priority_ops: str = Form(...)
):
    # Save uploaded file
    suffix = os.path.splitext(file.filename)[1]
    with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as temp:
        temp.write(await file.read())
        temp.flush()
        temp_path = temp.name

    # Run processor
    result = process_file(
        temp_path,
        json.loads(operations),
        json.loads(priority_ops)
    )

    # Save output file
    file_id = f"{uuid.uuid4()}.xlsx"
    output_path = f"processed_files/{file_id}"
    first_sheet = list(result["data"].keys())[0]
    result["data"][first_sheet].to_excel(output_path, index=False)

    return {
        "status": "completed",
        "file_id": file_id,
        "sheet": first_sheet,
        "preview": result["preview"],
        "operations_performed": list(json.loads(operations).keys())
    }

@app.get("/history")
def get_history():
    files = []
    for f in os.listdir("processed_files"):
        if f.endswith(".xlsx"):
            files.append({
                "id": f,
                "name": f,
                "created_at": os.path.getmtime(f"processed_files/{f}")
            })
    return files

@app.get("/download/{file_id}")
def download(file_id: str):
    return FileResponse(f"processed_files/{file_id}")
